This notebook demonstrates the use of the Pandeia engine with the NIRCam coronagraphs. Specifically, it provides examples of:
* Scene construction and instrument configuration
* Engine calculations (using the bundled precomputed PSF library)
* Basic data reduction with PSF subtraction
* Iterating over instrument configurations to optimize observations
* Contrast calculations

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from pandeia.engine.calc_utils import build_default_calc
from pandeia.engine.perform_calculation import perform_calculation

import jwst_pancake as pancake

from copy import deepcopy
import numpy as np

# Constructing a Scene

We'll start by defining the source and instrument properties for our desired observation. 

In [ ]:
target_mV =10.
ref_mV = 9.

target_Sp = 'a5v'
ref_Sp = 'a3v'

subarray = 'sub640'
filter_c = 'f210m'
mask_c = 'mask210r'

ngroup = 20
nint = 1

Now we'll load in a NIRCam template and configure the instrument for our observation.

In [ ]:
# Load the template
config = build_default_calc('jwst', 'nircam', 'coronagraphy')

# Set the coronagraph and filter
config['configuration']['detector']['subarray'] = subarray
config['configuration']['detector']['ngroup'] = ngroup
config['configuration']['instrument']['aperture'] = mask_c
config['configuration']['instrument']['filter'] = filter_c

This template contains a scene with a single star. We'll set the star properties and then duplicate it to create a planetary companion.

In [ ]:
# Pull out the target (the first entry in the 'scene' list)
target = config['scene'][0]
target['spectrum']['normalization'] = {'type': 'photsys', 'norm_fluxunit': 'abmag', 'bandpass': 'johnson,j'}
target['spectrum']['normalization']['norm_flux'] = target_mV
target['spectrum']['sed'] = {'sed_type': 'phoenix'}
target['spectrum']['sed']['key'] = target_Sp
target['id'] = 1

# Copy the target star and turn it into a planet
planetA = deepcopy(target)
planetA['id'] = 2 #each source must have a unique ID, starting at 1

# A different way to normalize source flux
planetA['spectrum']['normalization']['bandpass'] = 'nircam,sw_imaging,f210m'
planetA['spectrum']['normalization']['norm_flux'] = 16.5
planetA['spectrum']['normalization']['type'] = 'jwst'
planetA['spectrum']['sed']['sed_type'] = 'blackbody'
planetA['spectrum']['sed']['temp'] = 900.
del planetA['spectrum']['sed']['key'] #unnecessary now

# Source offset
planetA['position']['x_offset'] = 0.406 #arcsec
planetA['position']['y_offset'] = -1.263

# Copy that planet and turn it into a second one
planetB = deepcopy(planetA)
planetB['id'] = 3
planetB['position']['x_offset'] = -0.306 #arcsec
planetB['position']['y_offset'] = -.53

# Update calculation file with the new planet
config['scene'].extend([planetA,planetB])

Rotate the scene

In [ ]:
pancake.scene.rotate_scene(config['scene'],35.,center=[0.,0.])

Now we'll add in a global offset to the entire scene to capture the effect of target acquisition error.

In [ ]:
errx, erry = pancake.scene.get_ta_error()
pancake.scene.offset_scene(config['scene'],errx,erry)

And now for the reference scene for PSF subtraction:

In [ ]:
reference = config['strategy']['psf_subtraction_source']

# We adopt a brighter but spectrally-mismatched reference
reference['id'] = 99
reference['spectrum'] = deepcopy(target['spectrum'])
reference['spectrum']['normalization']['norm_flux'] = ref_mV
reference['spectrum']['sed']['key'] = ref_Sp

# And add target acquisition error
errx_ref, erry_ref = pancake.scene.get_ta_error()
pancake.scene.offset_scene([reference],errx_ref,erry_ref)

And now we'll plot the two scenes we've constructed

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121,polar=True)
pancake.scene.plot_scene(config['scene'],'Target Scene w/ Companion',newfig=False)
ax = plt.gca()
ax.set_rlim(0,5.)
plt.subplot(122,polar=True)
pancake.scene.plot_scene([reference],'Reference Scene',newfig=False)
ax = plt.gca()
ax.set_rlim(0,5.)

# Run the Pandeia Engine

Now we pass our calculation files to the pandeia engine to create the slope images (and a number of other products).

The most direct way to do so is to pass the calculation file to ```pancake.engine.perform_calculation.``` However, in order to get both the target and the reference spectrum individually, the convenience function ```pancake.engine.calculate_all``` will calculate both the target and reference in a single function (and, in pandeia 1.3 and above, in a single calculation).

### Wave Sampling

An aside on performance and accuracy: The ```engine.options.wave_sampling``` parameter provides a hook into the wavelength sampling of the 3D (x,y,wavelength) cube. By default, Pandeia adopts some large value for the wavelength sampling (typically 150+); however, this is the primary time sink in the calculation. Setting ```engine.options.wave_sampling = 10``` while developing your simulation provides dramatic time savings while getting within ~5% of the "true" value. By ```engine.options.wave_sampling = 40```, one can expect agreement to within roughly 1%.

### On-the-fly PSF Calculations

The Pandeia engine relies on a library of precomputed PSFs that are sparsely sampled across the coronagraphic field of view. For some observing strategies or coronagraphs, this sparse sampling will often be insufficient for accurately capturing PSF variations arising from small offsets.

Pandeia-Coronagraphy gives the option (```engine.options.on_the_fly_PSFs```) to circumvent the use of this precomputed library and force recomputing each PSF on the fly in WebbPSF.

NIRCam is, in general, less sensitive to spatial variations, so we won't bother toggling it to True here. See the notebooks with small grid dithers or MIRI for examples of on-the-fly PSF calculations.

In [ ]:
results = pancake.engine.calculate_all(config)

targ_results = results['target']
ref_results = results['reference']

Plot the detector images (in e$^-$/s)

In [ ]:
target_slope = targ_results['2d']['detector']
reference_slope = ref_results['2d']['detector']

plt.figure(figsize=(10,4))
plt.subplot(121)
plt.imshow(target_slope)
plt.title('Target Slope Image')
plt.colorbar().set_label('e$^{-}$/s')
plt.subplot(122)
plt.imshow(reference_slope)
plt.title('Reference Slope Image')
plt.colorbar().set_label('e$^{-}$/s')

And check for saturation:

In [ ]:
target_sat = targ_results['2d']['saturation']
reference_sat = ref_results['2d']['saturation']

plt.figure(figsize=(10,4))
plt.subplot(121)
plt.imshow(target_sat)
plt.title('Target Saturation Image')
plt.subplot(122)
plt.imshow(reference_sat)
plt.title('Reference Saturation Image')

# Post-Processing

Subtract the registered and scaled reference PSF from the target image.

In [ ]:
# Mean-subtract the two images (and handle any NaNs)
centered_target = target_slope - np.nanmean(target_slope)
centered_reference = reference_slope - np.nanmean(reference_slope)
centered_reference[np.isnan(centered_reference)] = np.nanmax(centered_reference)

# Register the reference to the target and renormalize
registered_ref = pancake.analysis.register_to_target(centered_reference,centered_target)

In [ ]:
plt.imshow(centered_target - registered_ref)
plt.colorbar().set_label('e$^{-}$/s')
plt.title('After Reference-Subtraction')

# Optimizing Observation Parameters


In [ ]:
ngroup_list = range(2,8)

raw_images = []
refsub_images = []
for ngroup in ngroup_list:    
    #Exposure parameters
    numberofgroups = ngroup
    numberofints = 1
    current_config = deepcopy(config)
    current_config['configuration']['detector']['ngroup'] = numberofgroups
    current_config['configuration']['detector']['nint'] = numberofints

    #Pandeia calculation
    results = pancake.engine.calculate_all(current_config)
    occ_results = results['target']
    ref_results = results['reference']
    occ_slope = occ_results['2d']['detector']
    ref_slope = ref_results['2d']['detector']

    #PSF subtraction assuming photon noise, the normalization is done properly 
    centered_occ = occ_slope - np.nanmean(occ_slope)
    centered_ref = ref_slope - np.nanmean(ref_slope)
    reg_ref = pancake.analysis.register_to_target(centered_ref,centered_occ,rescale_reference=True)
    ref_sub = centered_occ - reg_ref
    
    raw_images.append(centered_occ)
    refsub_images.append(ref_sub)

    # Uncomment to provide an indication of how quickly the calculation is proceeding
    # print("Finished Iteration {} of {}".format(len(raw_images), len(ngroup_list)))

In [ ]:
for raw, reduced in zip(raw_images,refsub_images):
    plt.figure(figsize=(10,4))
    plt.subplot(121)
    plt.imshow(raw)
    plt.title('Raw Data')
    plt.colorbar().set_label('e$^{-}$/s')
    plt.subplot(122)
    plt.imshow(reduced)
    plt.title('Reduced Data')
    plt.colorbar().set_label('e$^{-}$/s')

# Save Calculation File

Save out your scene and instrument parameters for quick loading with a future call to ```engine.load_calculation```

In [ ]:
pancake.engine.save_calculation(target,'mygreatcalculation.json')

# Save Pandeia Images

```pancake.engine.save_to_fits``` is provided as a convenience function for quickly saving out arrays or cubes to a FITS file. This doesn't preserve any header values. See http://docs.astropy.org/en/stable/io/fits/ for a more complete treatment of reading and writings FITS files in Python.

In [ ]:
# Save out 2d slop images
pancake.engine.save_to_fits(targ_results['2d']['detector'],'target_slope.fits')
pancake.engine.save_to_fits(ref_results['2d']['detector'],'reference_slope.fits')

# Save out cube
pancake.engine.save_to_fits(raw_images,'raw_cube.fits')

# Limiting Contrast Calculation


This is where we calculate the contrast as a function of exposure time. Note that in this example the limiting contrast is driven by:

1. detector noise, in the Pandeia model
2. photon noise on the speckles
3. TA error between target and reference. 

There is no error due to wavefront thermal drifts or dynamical vibrations 

For the occulted source, we'll copy the target calculation from before and pop the planet out of the scene list.

In [ ]:
occulted = deepcopy(config)
occulted['scene'].pop(-1)
occulted['scene'].pop(-1)

#and plot
pancake.scene.plot_scene(occulted['scene'],'Occulted Star')
ax = plt.gca()
ax.set_rlim(0,1.5)

The unocculted source for the contrast normalization is just the target source moved from behind the coronagraphic mask.

In [ ]:
#copy the occulted calculation file
unocculted = deepcopy(occulted)
#apply an offset
unocculted['scene'][0]['position']['x_offset'] = 0.8 # arcsec
unocculted['scene'][0]['position']['y_offset'] = 0.8 # arcsec

unocculted['calculation']['effects']['saturation'] = False # Disable saturation

#and plot
pancake.scene.plot_scene(unocculted['scene'],'Unocculted Star')
ax = plt.gca()
ax.set_rlim(0,1.5)

 We'll use the same reference as in the previous calculations.

In [ ]:
from scipy.signal import fftconvolve

def quick_contrast(occulted_image,unocculted_image,n_annuli=20):
    '''
    A quick and dirty contrast calculation just for demonstration
    purposes.
    '''
    # Convolve the unocculted image with an aperture and pick out the max 
    # as the normalization constant
    kernel = np.array([[0,0,1,0,0], #simple aperture
                   [0,1,1,1,0],
                   [1,1,1,1,1],
                   [0,1,1,1,0],
                   [0,0,1,0,0]]).astype(float)
    unocc_aperture = fftconvolve(unocculted_image,kernel,mode='valid')
    norm = np.max(unocc_aperture)

    # Convolve reference-subtract and raw frames with the aperture as well
    occ_aperture = fftconvolve(occulted_image,kernel,mode='valid')

    # Compute radial distance from center (in pixels)
    indices = np.indices(unocc_aperture.shape)
    center = np.array(unocc_aperture.shape) / 2.
    radial = np.sqrt( (indices[0] - center[0])**2 + (indices[1] - center[1])**2 )
    # Compute 20 annuli (uniform in radius)
    radial_bins = np.linspace(0,np.max(radial),num=n_annuli)
    annuli_inds = np.digitize(radial,radial_bins)

    # Take the variance of raw and reference-subtracted images in each annulus and normalize by unocculted max
    contrast = np.array([np.std(occ_aperture[annuli_inds == a]) for a in np.unique(annuli_inds)]) / norm    
    return radial_bins, contrast

Now we'll iterate over exposure parameters and calculate the raw and reference-subtracted contrast at each iteration.

In [ ]:
n_annuli = 30
ngroup_list = range(2,11)

raw_contrast_list = []
refsub_contrast_list = []
for ngroup in ngroup_list:
    # Exposure parameters
    numberofgroups = ngroup
    numberofints = 1
    occulted = deepcopy(config)
    occulted['configuration']['detector']['ngroup'] = numberofgroups
    occulted['configuration']['detector']['nint'] = numberofints
    unocculted = deepcopy(config)
    unocculted['configuration']['detector']['ngroup'] = 200
    unocculted['configuration']['detector']['nint'] = 10

    # Pandeia calculation
    occulted_results = pancake.engine.calculate_all(occulted)
    occ_results = occulted_results['target']
    ref_results = occulted_results['reference']
    unocc_results = pancake.engine.calculate_contrast(unocculted, 0.8, 0.8)
    occ_slope = occ_results['2d']['detector']
    unocc_slope = unocc_results['2d']['detector']
    ref_slope = ref_results['2d']['detector']

    # PSF subtraction assuming photon noise, the normalization is done properly 
    centered_occ = occ_slope - np.nanmean(occ_slope)
    centered_ref = ref_slope - np.nanmean(ref_slope)
    reg_ref = pancake.analysis.register_to_target(centered_ref,centered_occ)
    ref_sub = centered_occ - reg_ref

    radial_bins, raw_contrast = quick_contrast(occ_slope,unocc_slope,n_annuli=n_annuli)
    radial_bins, refsub_contrast = quick_contrast(ref_sub,unocc_slope,n_annuli=n_annuli)

    raw_contrast_list.append(raw_contrast)
    refsub_contrast_list.append(refsub_contrast)
    
    # Uncomment to get feedback as the calculations run
    # print("Finished iteration {} of {}".format(len(raw_contrast_list), len(ngroup_list)))

In [ ]:
pix_scale = 0.032 # see https://jwst-docs.stsci.edu/display/JTI/NIRCam+Imaging
mask_radii = 0.4 # see https://jwst-docs.stsci.edu/display/JTI/NIRCam+Coronagraphic+Imaging

plt.figure(figsize=(10,5))
for i,contrast in enumerate(raw_contrast_list):
    plt.semilogy(radial_bins * pix_scale,contrast,label='Nints=1, Ngroups={}'.format(ngroup_list[i]))
    plt.fill_between([0,mask_radii],1e-6,20,alpha=0.2,lw=0,color=[0.5,0.5,0.5])
    plt.xlabel('Radial Separation (arcsec)')
    plt.ylabel('Contrast')
    plt.xlim(0,2.1)
    plt.ylim(1e-6,1e-2)
    plt.title('Raw Target, {} V={}, Reference {}, V={}'.format(target_Sp,target_mV,ref_Sp,ref_mV),fontsize=15)
    plt.legend(loc='upper right')
    
plt.figure(figsize=(10,5))
for i,contrast in enumerate(refsub_contrast_list):
    plt.semilogy(radial_bins * pix_scale,contrast,label='Nints=1, Ngroups={}'.format(ngroup_list[i]))
    plt.fill_between([0,mask_radii],1e-6,20,alpha=0.2,lw=0,color=[0.5,0.5,0.5])
    plt.xlabel('Radial Separation (arcsec)')
    plt.ylabel('Contrast')
    plt.xlim(0,2.1)
    plt.ylim(1e-6,1e-2)
    plt.title('Reduced Target, {} V={}, Reference {}, V={}'.format(target_Sp,target_mV,ref_Sp,ref_mV),fontsize=15)
    plt.legend(loc='upper right')
    